## Генерируем ответы для вопросов
Ищем информацию в базе знаний, а потом генерируем ответ по данным, которые мы знаем

# PROCESSING

In [1]:
import pandas as pd
import murgpt

# LOAD

In [2]:
data = pd.read_csv('data_dozor/DATAMAIN.csv')
data = data.drop('Unnamed: 0', axis=1)
data['embeddings'] = data.embeddings.apply(murgpt.emb_to_list) #делает эмбеддинги нормальными в пандас
data['embeddings_tl_dr'] = data.embeddings_tl_dr.apply(murgpt.emb_to_list) #делает эмбеддинги нормальными в пандас
data['embeddings_qa'] = data.embeddings_qa.apply(murgpt.emb_to_list) #делает эмбеддинги нормальными в пандас

In [3]:
data.head()

,title,content,count,tl_dr,tldr_count,questions,embeddings_tl_dr,embeddings,embeddings_qa
0,Общая информация о 'Дозор',Дозор. Мониторинг систем безопасности рабочей...,592,"Программный продукт ""Дозор"" предназначен для ...",115,Какую информацию содержит программный продукт...,"[-0.005670938175171614, -0.0042793177999556065...","[-0.004247976001352072, -0.009159590117633343,...","[-0.003868026426061988, 0.00249838107265532, -..."
1,Назначение и основные возможности ПП Назначени...,Назначение и основные возможности ПП Назначени...,331,"Программный продукт ""Дозор. Мониторинг систем...",150,"Какие функции выполняет программный продукт ""...","[0.00029265889315865934, -0.010223266668617725...","[-0.003276773961260915, -0.012050516903400421,...","[0.00019895278092008084, 0.00657667126506567, ..."
2,Назначение и основные возможности ПП Модуль пр...,Назначение и основные возможности ПП Модуль пр...,102,"Описывается программный продукт ""Модуль прове...",65,Для чего нужен ПП Модуль проверки антивирусно...,"[-0.021989833563566208, -0.004621290601789951,...","[-0.012202619574964046, 0.00047983555123209953...","[0.006612807046622038, 0.005957254208624363, -..."
3,Назначение и основные возможности ПП Модуль пр...,Назначение и основные возможности ПП Модуль пр...,514,ПП Модуль проверки доступа к запрещенным сайт...,117,Какие возможности предоставляет ПП Модуль про...,"[-0.004668551497161388, -0.008102389052510262,...","[-0.0017716350266709924, -0.006506671197712421...","[0.009005137719213963, -0.006601388566195965, ..."
4,Назначение и основные возможности ПП Модуль пр...,Назначение и основные возможности ПП Модуль пр...,332,ПП Модуль проверки ПО на доверие ищет и отобр...,135,"""Какие возможности предлагает ПП Модуль прове...","[-0.008359338156878948, -0.011571301147341728,...","[0.0024968564976006746, -0.005762236658483744,...","[0.021388832479715347, 0.006385922431945801, -..."


# FINAL

In [4]:
QUESTION = "Как настроить параметры сканирования и отображения информации об установленном программном обеспечении?"

In [6]:
context = murgpt.create_context_2ver(QUESTION, data,                    # поиск информации для ответа
                                     emb_name='embeddings_tl_dr',       # поиск по кратким содержаниям
                                     max_len=1500,
                                     cont_name='content'
                                     )          
ANSWER  =  murgpt.generate_answer(QUESTION, context, temp=0)             # генерируем ответ 

In [7]:
print(ANSWER)

assistant: Для настройки параметров сканирования и отображения информации об установленном программном обеспечении в программе "Дозор" необходимо выполнить следующие действия:

1. Открыть главное окно программы "Дозор".
2. Нажать на кнопку "Настройки".
3. В открывшемся окне выбрать вкладку "Сканирование".
4. В разделе "Параметры сканирования" выбрать нужные опции, например, сканирование реестра, сканирование файловой системы и т.д.
5. В разделе "Отображение информации" выбрать нужные опции, например, отображение информации о версии программного обеспечения, отображение информации о дате установки и т.д.
6. Нажать кнопку "Сохранить настройки".

После выполнения этих действий программа "Дозор" будет сканировать и отображать информацию об установленном программном обеспечении в соответствии с выбранными параметрами.


In [7]:
#print(context)

### Создадим чат-алгоритм модуль для запоминания контекста 

In [18]:
def run_diolog(QUESTION, curr_c, add_mes=[]):
    context_help = murgpt.create_context_3ver(QUESTION, data,                # поиск информации для ответа
                                     emb_name='embeddings',
                                     max_len=500,
                                     cont_name='content',
                                     threshold = 0.25
                                     ) 
    
    ANSWER  =  murgpt.generate_answer(QUESTION, context_help, temp=0,
                                      curr_count = curr_c,
                                      mes_add = add_mes
                                      )
    
    return ANSWER, context_help

In [19]:
curr_ans = 0
add_mes = []
fr = True

while True:
    if fr:
        print("Привет, я помогу разобраться с ПП 'Дозор', Задайте свой вопрос! >>>:")
    fr = False
    
    str_in = input()
    if str_in == "1":
        break

    answer, context = run_diolog(str_in, curr_ans%3, add_mes)
    print('ANSWER:', answer)

    curr_ans+=1

    # теперь запоминаем контекс (каждые три те память длиной в три ответа)
    if curr_ans % 3 != 0:
        add_mes.append({"role": "user", "content": str_in})
        add_mes.append({"role": "assistant", "content": context})
    else:
        add_mes = []

    print('add:', add_mes)


Привет, я помогу разобраться с ПП 'Дозор', Задайте свой вопрос! >>>:
ANSWER: assistant: Для установки программы на Linux необходимо выполнить следующие шаги:

1. Скачать установочный файл программы с официального сайта или из другого источника.

2. Открыть терминал и перейти в папку, где находится скачанный файл.

3. Выполнить команду для установки программы, например: `sudo dpkg -i имя_файла.deb`.

4. После установки запустить программу с правами администратора, используя ярлык запуска в папке с установленным ПП или команду запуска сценария.

5. Если программа требует активации, необходимо выполнить ее в окне «Настройки», перейдя в которое из главного окна ПП, выбрав вкладку «Активация» и указав путь к файлу с действительной лицензией.

После выполнения всех этих шагов программа будет готова к работе.
add: [{'role': 'user', 'content': 'как установить '}, {'role': 'assistant', 'content': 'Установка программы Активация программного продукта на Linux\n После установки ПП на компьютер – р